# How to stream
- https://langchain-ai.github.io/langgraph/how-tos/streaming/

`graph.stream(..., stream_mode=<stream_mode>)`

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START


class State(TypedDict):
    topic: str
    joke: str


def refine_topic(state: State):
    return {"topic": state["topic"] + " and cats"}


def generate_joke(state: State):
    return {"joke": f"This is a joke about {state['topic']}"}


graph = (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "generate_joke")
    .compile()
)

### Stream all values in state with `stream_mode="values"`

In [2]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode="values"
):
    print(chunk)

{'topic': 'ice cream'}
{'topic': 'ice cream and cats'}
{'topic': 'ice cream and cats', 'joke': 'This is a joke about ice cream and cats'}


### Stream state updates from nodes with `stream_mode="updates"`

In [3]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode="updates"
):
    print(chunk)

{'refine_topic': {'topic': 'ice cream and cats'}}
{'generate_joke': {'joke': 'This is a joke about ice cream and cats'}}


### Stream debug events with `stream_mode="debug"`

In [4]:
for chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode="debug"
):
    print(chunk)

{'type': 'task', 'timestamp': '2025-03-04T08:12:00.302236+00:00', 'step': 1, 'payload': {'id': 'a60ef441-2164-7f96-f8ef-c28ed0dec121', 'name': 'refine_topic', 'input': {'topic': 'ice cream'}, 'triggers': ['start:refine_topic']}}
{'type': 'task_result', 'timestamp': '2025-03-04T08:12:00.302469+00:00', 'step': 1, 'payload': {'id': 'a60ef441-2164-7f96-f8ef-c28ed0dec121', 'name': 'refine_topic', 'error': None, 'result': [('topic', 'ice cream and cats')], 'interrupts': []}}
{'type': 'task', 'timestamp': '2025-03-04T08:12:00.302632+00:00', 'step': 2, 'payload': {'id': 'f71b9045-5936-ae30-4b9c-33895562fa10', 'name': 'generate_joke', 'input': {'topic': 'ice cream and cats'}, 'triggers': ['refine_topic']}}
{'type': 'task_result', 'timestamp': '2025-03-04T08:12:00.302793+00:00', 'step': 2, 'payload': {'id': 'f71b9045-5936-ae30-4b9c-33895562fa10', 'name': 'generate_joke', 'error': None, 'result': [('joke', 'This is a joke about ice cream and cats')], 'interrupts': []}}


### Stream LLM tokens with `stream_mode="messages"`:

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

def generate_joke(state: State):
    llm_response = llm.invoke([{"role": "user", "content": f"Generate a joke about {state['topic']}"}])
    return {"joke": llm_response.content}

graph =  (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "generate_joke")
    .compile()
)

In [6]:
for message_chunk, metadata in graph.stream(
    {"topic": "ice cream"},
    stream_mode="messages"
):
    if message_chunk.content:
        print(message_chunk.content, end="|", flush=True)

Why| did| the| cat| sit| on| the| ice| cream| cone|?

|Because| it| wanted| to| be| a| p|urr|-f|ect| scoop|!| |

In [7]:
metadata

{'langgraph_step': 2,
 'langgraph_node': 'generate_joke',
 'langgraph_triggers': ['refine_topic'],
 'langgraph_path': ('__pregel_pull', 'generate_joke'),
 'langgraph_checkpoint_ns': 'generate_joke:a35ee441-d055-5cbc-9272-87755f134ba3',
 'checkpoint_ns': 'generate_joke:a35ee441-d055-5cbc-9272-87755f134ba3',
 'ls_provider': 'openai',
 'ls_model_name': 'gpt-4o-mini',
 'ls_model_type': 'chat',
 'ls_temperature': None}

### Stream custom data with `stream_mode="custom"`

In [8]:
from langgraph.types import StreamWriter

def generate_joke(state: State, writer: StreamWriter):
    writer({"custom_key": "Writing custom data while generating a joke"})
    return {"joke": f"This is a joke about {state['topic']}"}

graph = (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "generate_joke")
    .compile()
)

In [9]:
for chunk in graph.stream({"topic": "ice cream"}, stream_mode="custom"):
    print(chunk)

{'custom_key': 'Writing custom data while generating a joke'}


### Configure multiple streaming modes

In [11]:
for stream_mode, chunk in graph.stream(
    {"topic": "ice cream"},
    stream_mode=["updates", "custom"]
):
    print(f"Stream mode: {stream_mode}")
    print(chunk)
    print("\n")

Stream mode: updates
{'refine_topic': {'topic': 'ice cream and cats'}}


Stream mode: custom
{'custom_key': 'Writing custom data while generating a joke'}


Stream mode: updates
{'generate_joke': {'joke': 'This is a joke about ice cream and cats'}}


